In [31]:
import pandas as pd
import anndata
import seaborn as sns
import matplotlib.pyplot as plt
from ALLCools.plot import *
from ALLCools.mcds import MCDS
import numpy as np
import scanpy as sc
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import ranksums, kruskal
from concurrent.futures import ProcessPoolExecutor, as_completed
from statsmodels.sandbox.stats.multicomp import multipletests as FDR
from collections import defaultdict
import pingouin as pg

In [32]:
# Oligo NN in all slices
ct= 'OPC NN'
meta = pd.read_hdf('/data/male-amb/MERFISH/MERFISH.TotalNucleiMetadata.Rotation.BasicFilter.hdf')

In [33]:
annot = pd.read_hdf('/data/male-amb/MERFISH/MERFISH.integrated.01.hdf' )
annot = annot[annot['final_annot'] == ct]
annot['sample'] = annot.index.map(lambda x: '-'.join(x.split('-')[:-1]))
annot['standard_center_x'] = meta['standard_center_x']
annot['standard_center_y'] = meta['standard_center_y']
annot['volume'] = meta['volume']
annot['cell_id'] = annot.index
# drop slice C13
annot = annot[annot['slice'] != 'C13']
annot.head()

,volume,z,fov,experiment,region,slice,age,sample,n_counts,n_genes,...,y,L1,L2,L2_transfer,merf_clust,merf_clust_tt,final_annot,standard_center_x,standard_center_y,cell_id
cell,,,,,,,,,,,,,,,,,,,,,
C2-2mo-7_3,1404.281494,4,7,202309011129_aging-mouse-2mo-slice-25_VMSC01101,region_0,C2,2mo,C2-2mo,154.0,89.0,...,2719.615967,11.0,178.0,OPC NN,r1_17_1,r1_17_1,OPC NN,328.105835,2719.615967,C2-2mo-7_3
C2-2mo-7_67,586.273438,3,7,202309011129_aging-mouse-2mo-slice-25_VMSC01101,region_0,C2,2mo,C2-2mo,77.0,52.0,...,2723.599365,11.0,178.0,OPC NN,r1_17_1,r1_17_1,OPC NN,330.535583,2723.599365,C2-2mo-7_67
C2-2mo-8_34,1190.410400,7,8,202309011129_aging-mouse-2mo-slice-25_VMSC01101,region_0,C2,2mo,C2-2mo,85.0,57.0,...,3075.285645,11.0,178.0,OPC NN,r1_17_0,r1_17_0,OPC NN,384.611847,3075.285645,C2-2mo-8_34
C2-2mo-8_42,1087.061523,7,8,202309011129_aging-mouse-2mo-slice-25_VMSC01101,region_0,C2,2mo,C2-2mo,139.0,73.0,...,3095.301514,11.0,178.0,OPC NN,r1_17_0,r1_17_0,OPC NN,337.836426,3095.301514,C2-2mo-8_42
C2-2mo-12_3,1003.518066,7,12,202309011129_aging-mouse-2mo-slice-25_VMSC01101,region_0,C2,2mo,C2-2mo,100.0,64.0,...,3389.481445,11.0,178.0,OPC NN,r1_17_0,r1_17_0,OPC NN,541.885254,3389.481445,C2-2mo-12_3


In [34]:
# balance the number of cells in each sample and keep the index
np.random.seed(0)
sample_cells = annot['sample'].value_counts().min()
annot = annot.groupby('sample').apply(lambda x: x.sample(sample_cells)).reset_index(drop=True)
annot = annot.set_index('cell_id')
annot['sample'].value_counts()

/var/tmp/ipykernel_1087742/2548341249.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annot = annot.groupby('sample').apply(lambda x: x.sample(sample_cells)).reset_index(drop=True)


sample
C11-18mo          244
C11-2mo           244
C8-2mo            244
C8-18mo           244
C5-9mo            244
C5-2mo            244
C5-18mo           244
C3-9mo            244
C3-2mo            244
C3-18mo           244
C2-9mo            244
C2-2mo            244
C2-18mo           244
C12-9mo-repeat    244
C12-2mo           244
C12-18mo          244
C11-9mo-repeat    244
C8-9mo            244
Name: count, dtype: int64

In [35]:
# get a dict with keys as the Cxxx and values as the slices, order by 2mo, 9mo and 18mo
all_samples = annot['sample'].unique()
sample_dict = defaultdict(list)
for sample in all_samples:
    _slice = sample.split('-')[0]
    sample_dict[_slice].append(sample)
    # order the list by 2mo, 9mo and 18mo
    sample_dict[_slice].sort(key=lambda x: int(x.split('-')[1][:-2]))

# order the slices by number small to large
sample_dict = dict(sorted(sample_dict.items(), key=lambda x: int(x[0][1:])))
# remove C13
#sample_dict.pop('C13')
#sample_dict

In [36]:
# age DEGs at differnet slices
var_dim = 'gene'
merfish_ds = MCDS.open('/data/male-amb/MERFISH/MERFISH.TotalNucleiCell.BasicFilter.zarr', var_dim=var_dim, use_obs = annot.index)
merfish_ds

<xarray.MCDS>
Dimensions:      (cell: 4178, gene: 500)
Coordinates:
    blank_count  (cell) uint32 ...
  * cell         (cell) <U23 'C2-2mo-7_67' 'C2-2mo-8_34' ... 'C5-9mo-1149_35'
  * gene         (gene) <U13 'Klf6' 'Pdgfra' 'Sox9' ... 'AC125126.1' 'Aqp4'
    n_counts     (cell) uint32 ...
    n_genes      (cell) uint32 ...
Data variables:
    count        (cell, gene) uint32 dask.array<chunksize=(4178, 100), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  gene

In [37]:
adata = merfish_ds.get_count_adata(da_name='count', loading_chunk=50000)
annot = annot.loc[adata.obs_names]
adata.obsm['X_spatial_rotated'] = np.array(annot[['standard_center_x','standard_center_y']])
adata.obs['volume'] = annot['volume']
adata.obs['sample'] = annot['sample']
adata

Loading chunk 0-4178/4178


AnnData object with n_obs × n_vars = 4178 × 500
    obs: 'blank_count', 'n_counts', 'n_genes', 'volume', 'sample'
    obsm: 'X_spatial_rotated'

In [38]:
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=5)

In [39]:
# adata.var["mean"] = adata.X.mean(axis=0).A1
# adata.var["std"] = (adata.X.multiply(adata.X)).mean(axis=0).A1 - (adata.var["mean"].values ** 2)
# print(adata.var["std"].min())

In [40]:
# Normalizing to median total counts
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)
# Logarithmize the data


In [41]:
def diff_ranksum(c1, c2, data):
    #global data
    # print(c1, c2)
    fc = np.log2((data[c1].mean(axis=0) + 0.01) / (data[c2].mean(axis=0) + 0.01))
    pv = np.zeros(len(fc))
    for i in range(len(pv)):
        pv[i] = ranksums(data[c1][:, i], data[c2][:, i])[1]
    fdr = FDR(pv, 0.01, "fdr_bh")[1]
    #fdr = pv
    return [c1, c2, fc, fdr]
    

In [42]:
def get_deg_stats(tmp_adata, leg):
    expr = pd.DataFrame([tmp_adata.X[tmp_adata.obs['sample']==xx].mean(axis=0).A.ravel() for xx in leg], index=leg, columns=tmp_adata.var.index).T
    expr = expr[leg]
    
    data = {xx:tmp_adata.X[tmp_adata.obs['sample']==xx].toarray() for xx in leg}
    genefilter = (tmp_adata.X[tmp_adata.obs['sample'].isin(leg)].getnnz(axis=0) > 10)

    stats = []
    for i in range(tmp_adata.shape[1]):
        if genefilter[i]:
            tmp = [data[ct_region_age][:,i] for ct_region_age in leg]
            xx, yy = kruskal(*tmp)
            stats.append([xx, yy])
    stats = np.array(stats)
    stats[:, 1] = FDR(stats[:, 1], 0.01, "fdr_bh")[1]
    stats = pd.DataFrame(stats, columns=['stats', 'fdr'], index=tmp_adata.var.index[genefilter])

    stats = pd.concat([stats, expr], axis = 1).sort_values('fdr')

    cpu = 32
    with ProcessPoolExecutor(cpu) as executor:
        futures = []
        for c1 in range(len(leg)-1):
            for c2 in range(c1+1, len(leg)):
                future = executor.submit(
                    diff_ranksum,
                    c1=leg[c1],
                    c2=leg[c2],
                    data = data,
                )
                futures.append(future)

        result = []
        for future in as_completed(futures):
            result.append(future.result())
            # c1, c2 = result[-1][0], result[-1][1]
            # print(f'{leg[c1]} vs {leg[c2]} finished')

    for xx in result:
        stats[f"{xx[0]}.{xx[1]}.fdr"] =xx[3]
        #stats[f"{xx[0]}.{xx[1]}.log2fc"] =xx[2]
    
    return stats
        

In [43]:
all_results = {}

for _slice, leg in sample_dict.items():
    tmp_adata = adata[adata.obs['sample'].isin(leg)].copy()
    sc.pp.normalize_total(tmp_adata)
    sc.pp.log1p(tmp_adata)
    stats = get_deg_stats(tmp_adata, leg)
    all_results[_slice] = stats


In [44]:
import joblib
joblib.dump(all_results, f"result/balanced_{ct}_deg_result")

['result/balanced_OPC NN_deg_result']

## plot fdr

In [45]:
# from scipy.stats import zscore
# df_zscored = all_fc_df.apply(zscore, axis = 0)
# plot_deg_df = df_zscored.loc[common_degs]

In [46]:

# sum_abs = all_fc_df.abs().sum()
# abs_normalized_df = all_fc_df.div(sum_abs)
# plot_deg_df = abs_normalized_df.loc[common_degs]

In [47]:
# # apply fdr correction to each column
# all_fdr_df = FDR(all_fdr_df, 0.01, "fdr_bh")[1]
# all_fdr_df

In [48]:
# from scipy.stats import zscore
# df_zscored = all_fc_df.apply(zscore, axis = 0)
# plot_deg_df = df_zscored.loc[common_degs]

In [49]:

# sum_abs = all_fc_df.abs().sum()
# abs_normalized_df = all_fc_df.div(sum_abs)
# plot_deg_df = abs_normalized_df.loc[common_degs]

In [50]:
# # apply fdr correction to each column
# all_fdr_df = FDR(all_fdr_df, 0.01, "fdr_bh")[1]
# all_fdr_df